# Monte Carlo simulation on 3D Ising model

This is a 3D Ising model, extending from the 2D case (Ref. 2D Ising model) 


In [1]:
#----------------------------------------------------------------------#
#  Import modules
#----------------------------------------------------------------------#
import numpy as np


#----------------------------------------------------------------------#
#  Build system
#----------------------------------------------------------------------#
SIZE = 5
system = np.random.randint(2,size=(SIZE,SIZE,SIZE))
system[system==0] = -1

print "-"*50
print "\tThe initial spin order"
print "-"*50
print system
print "\n"

#----------------------------------------------------------------------#
#  Check periodic boundary conditions 
#----------------------------------------------------------------------#
def bc(i):
    if i+1 > SIZE-1:
        return 0
    if i-1 < 0:
        return SIZE-1
    else:
        return i

#----------------------------------------------------------------------#
#  Calculate energy (nearest interaction)
#----------------------------------------------------------------------#
J = 1
def energy(system, X, Y, Z):
    eng = -1 * system[X, Y, Z] * (
            J * (system[bc(X+1), Y, Z] + system[bc(X-1), Y, Z] +
                    system[X, bc(Y+1), Z] + system[X, bc(Y-1), Z] +
                    system[X, Y, bc(Z+1)] + system[X, Y, bc(Z-1)]))
    return eng

#----------------------------------------------------------------------#
#   Monte carlo loops
#----------------------------------------------------------------------#
STEPS = 1000
TEMP = 1

def main():
    for step, x in enumerate(range(STEPS)):
        X = np.random.randint(0,SIZE)
        Y = np.random.randint(0,SIZE)
        Z = np.random.randint(0,SIZE)

        E = -2. * energy(system, X, Y, Z)

        if E < 0.:
            system[X, Y, Z] *= -1
        elif np.exp(-1./TEMP*E) > np.random.rand():
            system[X, Y, Z] *= -1

print '='*50
print '\tThe final spin order'
print '='*50

main()
print system

--------------------------------------------------
	The initial spin order
--------------------------------------------------
[[[-1 -1  1  1 -1]
  [-1 -1 -1  1  1]
  [ 1 -1  1 -1 -1]
  [ 1 -1  1 -1 -1]
  [-1 -1  1  1 -1]]

 [[-1  1  1  1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1  1 -1  1]
  [ 1  1  1 -1 -1]
  [-1 -1 -1  1 -1]]

 [[-1  1 -1 -1  1]
  [ 1  1 -1  1  1]
  [-1 -1  1  1  1]
  [ 1  1 -1 -1  1]
  [ 1  1  1 -1 -1]]

 [[-1  1 -1  1 -1]
  [-1  1  1 -1  1]
  [ 1  1 -1  1 -1]
  [-1 -1 -1  1 -1]
  [-1  1  1 -1 -1]]

 [[-1 -1 -1  1 -1]
  [ 1 -1  1 -1  1]
  [-1  1 -1 -1  1]
  [ 1  1  1 -1  1]
  [ 1  1 -1 -1  1]]]


	The final spin order
[[[-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]]

 [[-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]]

 [[-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]]

 [[-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1]
  [-1 -1 -1 -1 -1